In [1]:
import pretty_midi
import os
import numpy as np
import heapq
import pickle

# See https://jazz-soft.net/demo/GeneralMidi.html for which instrument each number represents
instrument_numbers = [0, 6, 40, 41, 42, 43, 45, 60, 68, 70, 71, 73]
num_instruments = len(instrument_numbers)

num_notes = 128 # Number of pitches in MIDI

# We allow the network to step itself forward in time in increments of 10 ms.
# Thus the network can shift between 10 ms and 1 s, inclusive
num_time_shifts = 100 
min_time_shift = 0.01

# A message can be NOTE_ON, NOTE_OFF, TIME_SHIFT
message_dim = 2*num_notes + num_time_shifts

# quantize_time_shift: takes a time shift and puts it into the correct bin
# in our reduced representation
# ARGUMENTS
# time_shift: the number of seconds to shift
# RETURN: the quantized time shift
def quantize_time_shift(time_shift):
    return int(np.round(time_shift/min_time_shift)) - 1

# note_on_event: generates the index for a NOTE_ON event
# ARGUMENTS
# note: the MIDI number for the note to be played
# RETURN: the index for a NOTE_ON message
def note_on_event(note):
    return note

# note_off_event: generates the index for a NOTE_OFF event
# ARGUMENTS
# note: the MIDI number for the note to be turned off
# RETURN: the index for the NOTE_OFF message
def note_off_event(note):
    return num_notes + note

# time_shift_event: generates the index for a TIME_SHIFT event
# ARGUMENTS
# time_shift: the quantized time shift
# RETURN: the index of the TIME_SHIFT message
def time_shift_event(time_shift):
    assert(0 <= time_shift and time_shift < num_time_shifts)
    return 2*num_notes + time_shift

# append_time_shift: appends a time shift event to the data array. If the time
# shift is too large, split it into multiple small time shifts
# ARGUMENTS
# data: time shift events will be appended to this list
# time_shift: amount of time to shift
def append_time_shift(data, time_shift):
    time_shift = quantize_time_shift(time_shift)
    
    # Split large time shifts into multiple small time shifts
    while (time_shift >= num_time_shifts):
        data.append(time_shift_event(num_time_shifts - 1))
        time_shift -= (num_time_shifts - 1)

    if (time_shift >= 0):
        data.append(time_shift_event(time_shift))
    
base_path = 'musicnet_midis/'

fnum = 0 # Which file are we writing currently?

data_fnames = [] # Save file name corresponding to each numpy array

train = 0

for composer in os.listdir(base_path):
    print('Starting ' + composer)
    for fname in os.listdir(base_path + composer):
        try:
            mid = pretty_midi.PrettyMIDI(base_path + composer + '/' + fname)
        except:
            # There are 7 files that cause an IO error, both with mido and pretty_midi. Haven't looked into why
            continue
        
        # We'll save the final data in an Lx2 numpy array, where L is the total number of messages. Along the second dimension,
        # the first element is the message number, and the second element is the channel number
        
        # We also save an instruments array where element i contains the instrument program number associated with channel i
        instruments = []
        
        # First we store lists and convert to numpy arrays later
        data = []
        channels = []
        
        # Store the time of each message. These won't appear in the final data array
        times = []
        
        for i, instrument in enumerate(mid.instruments):          
            # Priority queue of notes to turn off and the times to turn them off.
            # Specifically, this is a list of tuples of the form (off_time, pitch),
            # where the first element of the list is always the next note to turn off
            off_queue = []
            
            time = 0
                        
            for n, note in enumerate(instrument.notes):
                # Fixes bug in 'Haydn/2104_op64n5_1.mid' where notes 674 and 675 are the same note
                if n > 0 and note.pitch == instrument.notes[n - 1].pitch and note.start == instrument.notes[n - 1].start:
                    continue
            
                # We need to turn off a note
                while off_queue and note.start > off_queue[0][0]:
                    data.append(note_off_event(off_queue[0][1]))
                    time = off_queue[0][0]
                    times.append(time)
                    heapq.heappop(off_queue)
                    
                time = note.start
                
                data.append(note_on_event(note.pitch))
                times.append(time)
                
                # Add this note to the queue of notes needing to be turned off
                heapq.heappush(off_queue, (note.end, note.pitch))
                
                if n == len(instrument.notes) - 1:
                    # No more notes left. Flush the off queue
                    while off_queue:
                        data.append(note_off_event(off_queue[0][1]))
                        time = off_queue[0][0]
                        times.append(time)
                        heapq.heappop(off_queue)
                        
            new_messages = len(data) - len(channels)
            channels = channels + [i for n in range(new_messages)]
            instruments.append(instrument.program)
            
            assert(len(times) == len(data))
        
        data = np.array(data, dtype=np.long)
        channels = np.array(channels, dtype=np.long)
        times = np.array(times, dtype=np.float)
        
        instruments = np.array(instruments, dtype=np.long)
        
        sort_idx = np.argsort(times, kind='mergesort')
        
        data = data[sort_idx]
        channels = channels[sort_idx]
        times = times[sort_idx]
        
        data_with_tshifts = []
        channels_with_tshifts = []
        
        for m in range(data.shape[0]):
            data_with_tshifts.append(data[m])
            channels_with_tshifts.append(channels[m])
            if m != data.shape[0] - 1 and times[m] != times[m + 1]:
                append_time_shift(data_with_tshifts, times[m + 1] - times[m])
                num_append = len(data_with_tshifts) - len(channels_with_tshifts)
                
                # We say that all time shifts occur on channel -1
                channels_with_tshifts = channels_with_tshifts + [-1 for i in range(num_append)]

        data_with_tshifts = np.expand_dims(np.array(data_with_tshifts), 1)
        channels_with_tshifts = np.expand_dims(np.array(channels_with_tshifts), 1)
        data = np.concatenate((data_with_tshifts, channels_with_tshifts), axis=1)
        
        # 80% train, 20% test
        if train == 4:
            folder = 'test'
        else:
            folder = 'train'
            
        np.save(folder + '_unified/recording' + str(fnum) + '.npy', data)
        np.save(folder + '_unified/instruments' + str(fnum) + '.npy', instruments)
        
        train = (train + 1)%5
        
        data_fnames.append(composer + '/' + fname)
        fnum += 1
        
pickle.dump(data_fnames, open( "preprocessed_data_unified_fnames.p", "wb" ) )

Starting Faure


/opt/conda/lib/python3.7/site-packages/pretty_midi/pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


Starting Schubert
Starting Cambini
Starting Ravel
Starting Bach
Starting Dvorak
Starting Brahms
Starting Mozart
Starting Beethoven
Starting Haydn


In [218]:
mid = pretty_midi.PrettyMIDI(base_path + 'Haydn/2104_op64n5_1.mid')
mid.instruments

[Instrument(program=40, is_drum=False, name="Violin 1"),
 Instrument(program=40, is_drum=False, name="Violin 2"),
 Instrument(program=41, is_drum=False, name="Viola"),
 Instrument(program=42, is_drum=False, name="Violoncello")]

674


In [255]:
print(mid.instruments[0].notes[674]) # 674 and 675 are the same note

Note(start=276.217216, end=276.309321, pitch=78, velocity=49)
